---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

In [3]:
def degree_distribution(G):
    degrees = G.degree()
    degree_vals = sorted(set(degrees.values()))
    hist = [list(degrees.values()).count(i) / float(nx.number_of_nodes(G)) for i in degree_vals]
    
    return hist

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
def graph_identification():
    
    methods = []
    for G in P1_Graphs:
        clustering = nx.average_clustering(G)
        shortest_path_length = nx.average_shortest_path_length(G)
        degree_hist = degree_distribution(G)
        if len(degree_hist)>10:
            methods.append('PA')
        elif clustering<0.1:
            methods.append('SW_H')
        else:
            methods.append('SW_L')
    
    return methods
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
def salary_predictions():
    
    def is_management(node):
        managementSalary = node[1]['ManagementSalary']
        if managementSalary == 0:
            return 0
        elif managementSalary == 1:
            return 1
        else:
            return None
    df = pd.DataFrame(index=G.nodes())
    
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))
    
    df['is_management'] = pd.Series([is_management(node) for node in G.nodes(data=True)])
    
    df_train = df[~pd.isnull(df['is_management'])]
    df_test = df[pd.isnull(df['is_management'])]
    
    features = ['clustering', 'degree', 'degree_centrality', 'closeness', 'betweeness', 'pr']
    
    X_train = df_train[features]
    Y_train = df_train['is_management']
    X_test = df_test[features]
    
    s = MinMaxScaler()
    X_train_scaled = s.fit_transform(X_train)
    X_test_scaled = s.fit_transform(X_test)
    
    clf = MLPClassifier()
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    
    return pd.Series(test_proba, X_test.index)
salary_predictions()

1       0.430818
2       0.989748
5       0.999994
8       0.282960
14      0.916978
18      0.674706
27      0.811474
30      0.794880
31      0.422418
34      0.280622
37      0.252851
40      0.637580
45      0.507270
54      0.557775
55      0.609586
60      0.556281
62      0.999995
65      0.996176
77      0.124426
79      0.294241
97      0.187919
101     0.128522
103     0.534739
108     0.336122
113     0.951789
122     0.111887
141     0.966089
142     0.998324
144     0.090753
145     0.892005
          ...   
913     0.109417
914     0.142686
915     0.007938
918     0.259452
923     0.077326
926     0.137989
931     0.095305
934     0.021735
939     0.020506
944     0.007785
945     0.067978
947     0.230319
950     0.256787
951     0.077257
953     0.054744
959     0.007504
962     0.008369
963     0.328352
968     0.169061
969     0.146177
974     0.115999
984     0.020242
987     0.152790
989     0.168523
991     0.164062
992     0.008406
994     0.007423
996     0.0077

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
def new_connections_predictions():
    
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
        
    preferential_attachment = list(nx.preferential_attachment(G))
    
    df = pd.DataFrame(index=[(i[0], i[1]) for i in preferential_attachment])
    df['preferential_attachment'] = [i[2] for i in preferential_attachment]
    
    csh = list(nx.cn_soundarajan_hopcroft(G))
    
    df_csh = pd.DataFrame(index=[(i[0], i[1]) for i in csh])
    df_csh['cn_soundarajan_hopcroft'] = [i[2] for i in csh]
                          
    df = df.join(df_csh, how='outer')
                          
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(value=0)
    
    df['resource_allocation_index'] = [i[2] for i in list(nx.resource_allocation_index(G))]
    df['jaccard_coefficient'] = [i[2] for i in list(nx.jaccard_coefficient(G))]
                          
    df = future_connections.join(df, how='outer')
    
    # Train & test                    
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
                          
    features = ['cn_soundarajan_hopcroft', 'preferential_attachment',
                'resource_allocation_index', 'jaccard_coefficient']
    
    X_train = df_train[features]
    Y_train = df_train['Future Connection']
    X_test = df_test[features]
    
    s = MinMaxScaler()
    X_train_scaled = s.fit_transform(X_train)
    X_test_scaled = s.fit_transform(X_test)
                          
    clf = MLPClassifier()
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]

    predictions = pd.Series(test_proba, X_test.index)
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['prob'] = [predictions[i] for i in target.index]
    
    return target['prob']
    
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:565: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.029972
(542, 751)    0.013232
(20, 426)     0.885565
(50, 989)     0.013243
(942, 986)    0.013255
(324, 857)    0.013241
(13, 710)     0.358278
(19, 271)     0.285344
(319, 878)    0.013247
(659, 707)    0.013233
(49, 843)     0.013250
(208, 893)    0.013235
(377, 469)    0.012754
(405, 999)    0.019673
(129, 740)    0.018209
(292, 618)    0.029157
(239, 689)    0.013247
(359, 373)    0.012956
(53, 523)     0.095574
(276, 984)    0.013250
(202, 997)    0.013252
(604, 619)    0.176461
(270, 911)    0.013247
(261, 481)    0.113894
(200, 450)    0.998451
(213, 634)    0.013228
(644, 735)    0.103730
(346, 553)    0.013200
(521, 738)    0.013145
(422, 953)    0.020490
                ...   
(672, 848)    0.013247
(28, 127)     0.991848
(202, 661)    0.013195
(54, 195)     0.999992
(295, 864)    0.013240
(814, 936)    0.013230
(839, 874)    0.013255
(139, 843)    0.013241
(461, 544)    0.013106
(68, 487)     0.013099
(622, 932)    0.013237
(504, 936)    0.019070
(479, 528) 